In [1]:
%load_ext autoreload

In [2]:
import os
import sys
from typing import Tuple, List, Tuple, Optional
import tempfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import joblib
import subprocess as sp
import time

import torch
from torch import nn
import torchvision.transforms as T
import torchvision
from torchvision.datasets import DatasetFolder, ImageFolder
from torchvision.models import resnet50
from torch.utils.data import ConcatDataset, DataLoader
import matplotlib.pyplot as plt
from collections import defaultdict

from torchvision.models import resnet18, resnet50
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

from sklearn.metrics import roc_curve, auc
from sklearn.decomposition import PCA as Sklearn_PCA

In [3]:
%autoreload 2
from fundus_extractor.utils.general import imshow, normalize_image
from fundus_extractor.utils.datasets import Fundus_Left_Right_Combined_Dataset
from fundus_extractor.models import pretrained_pca

In [4]:
def submit_slurm_array(training_save_dir: str, cmds: list, memory_in_gb: int, cores: int, gpu: bool, queue: str,
                       n_parallel_jobs: int, **kwargs) -> None:
    os.makedirs(training_save_dir, exist_ok=True)

    for ii, cmd in enumerate(cmds):
        job_dir = os.path.join(training_save_dir, f'run_{ii}') 
        os.makedirs(job_dir, exist_ok=True)
        job_file = f'{job_dir}/job.sh'
        out_file = f'{job_dir}/out.txt'
        err_file = f'{job_dir}/err.txt'
        with open(job_file, 'w') as handle:
            handle.writelines("#!/bin/bash\n")
            handle.writelines(f"source $HOME/.bashrc\n")
            handle.writelines(f"conda activate deepMMR\n")
            handle.writelines(f'{cmd} >> {out_file} 2>> {err_file}\n\n')
        os.chmod(job_file, 0o777)
        
    job_file = os.path.join(training_save_dir, 'job.sh')
    out_file = os.path.join(training_save_dir, 'out.txt')
    err_file = os.path.join(training_save_dir, 'err.txt')
    
    with open(job_file, 'w') as handle:
        handle.writelines("#!/bin/bash\n")

        handle.writelines('#SBATCH -J MMR_Job_Array\n')
        handle.writelines(f'#SBATCH -o {out_file}\n')
        handle.writelines(f'#SBATCH -e {err_file}\n')

        handle.writelines('#SBATCH -t 24:00:00\n')
        handle.writelines(f'#SBATCH -p {queue}\n')
        handle.writelines(f'#SBATCH -c {cores}\n')
        handle.writelines(f'#SBATCH --mem={memory_in_gb}GB\n')
        handle.writelines(f'#SBATCH --exclude=ouga[16-17]\n')
        if gpu is True:
            handle.writelines('#SBATCH --gres=gpu:1\n')
        handle.writelines(f'#SBATCH -a 0-{len(cmds)-1}%{n_parallel_jobs}\n')

        handle.writelines(f'cd {training_save_dir}\n')
        handle.writelines('./run_${SLURM_ARRAY_TASK_ID}/job.sh\n')

    sp.Popen(["sbatch", job_file], stdout=sp.PIPE, stderr=sp.PIPE)

# Setup

In [5]:
dataset_dir = '/s/project/deepMMR/data/fundus/processed/left'
dataset_left = Fundus_Left_Right_Combined_Dataset(dataset_dir, loader=lambda x: normalize_image(torchvision.io.read_image(x)))
train_dataloader_left = DataLoader(dataset_left, batch_size=1024)

dataset_dir = '/s/project/deepMMR/data/fundus/processed/right'
dataset_right = Fundus_Left_Right_Combined_Dataset(dataset_dir, loader=lambda x: normalize_image(torchvision.io.read_image(x)))
train_dataloader_right = DataLoader(dataset_right, batch_size=1024)

FileNotFoundError: [Errno 2] No such file or directory: '/s/project/deepMMR/data/fundus/processed/left'

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Device: {device}')

# Evaluation - transferGWAS

In [6]:
save_path_left = '/data/ouga/home/ag_gagneur/graef/tmp/retinas_left.csv'
save_path_right = '/data/ouga/home/ag_gagneur/graef/tmp/retinas_right.csv'
save_path = '/data/ouga/home/ag_gagneur/graef/tmp/retinas.csv'

In [7]:
data = defaultdict(list)

images_dir_left = '/s/project/deepMMR/data/fundus/processed_v_1/left/'
for sub_dir in os.listdir(images_dir_left):
    for file in os.listdir(os.path.join(images_dir_left, sub_dir)):
        eid, _, instance_id, group_id = file.split('.')[0].split('_')
        data['IID'].append('_'.join([eid, instance_id, group_id]))
        data['path'].append(os.path.join(images_dir_left, sub_dir, file))
        data['instance'].append('left')

images_dir_right = '/s/project/deepMMR/data/fundus/processed_v_1/right/'
for sub_dir in os.listdir(images_dir_right):
    for file in os.listdir(os.path.join(images_dir_right, sub_dir)):
        eid, _, instance_id, group_id = file.split('.')[0].split('_')
        data['IID'].append('_'.join([eid, instance_id, group_id]))
        data['path'].append(os.path.join(images_dir_right, sub_dir, file))
        data['instance'].append('right')

data_df = pd.DataFrame(data)
data_df = data_df.groupby('IID').filter(lambda x: len(x) == 2)
data_df

,IID,path,instance
0,1178447_0_0,/s/project/deepMMR/data/fundus/processed_v_1/l...,left
1,1186186_0_0,/s/project/deepMMR/data/fundus/processed_v_1/l...,left
2,1181999_0_0,/s/project/deepMMR/data/fundus/processed_v_1/l...,left
3,1178854_0_0,/s/project/deepMMR/data/fundus/processed_v_1/l...,left
4,1173877_0_0,/s/project/deepMMR/data/fundus/processed_v_1/l...,left
...,...,...,...
178270,4607590_0_0,/s/project/deepMMR/data/fundus/processed_v_1/r...,right
178271,4614645_1_0,/s/project/deepMMR/data/fundus/processed_v_1/r...,right
178272,4607023_0_0,/s/project/deepMMR/data/fundus/processed_v_1/r...,right
178273,4623261_1_0,/s/project/deepMMR/data/fundus/processed_v_1/r...,right


In [8]:
cmds = []
for ii, indices in enumerate(np.array_split(np.unique(data_df['IID']), 8)):
    save_dir = f'/data/ouga/home/ag_gagneur/graef/tmp/slurm/img_extraction/run_{ii}'
    os.makedirs(save_dir, exist_ok=True)
    indices_path = os.path.join(save_dir, 'retinas.csv')
    data_df[data_df['IID'].isin(indices)].to_csv(indices_path, sep=',', index=None)
    cmd = f'python /data/ouga/home/ag_gagneur/graef/Programs/transferGWAS/feature_condensation/feature_condensation.py {indices_path} {save_dir} \
                    --tfms tta \
                    --dev cuda:0 \
                    --n_pcs 256 \
                    --model resnet50 \
                    --pretraining imagenet \
                    --layer L2 L3 L4'
    cmds.append(cmd)
submit_slurm_array('/data/ouga/home/ag_gagneur/graef/tmp/slurm/img_extraction', cmds, 32, 16, True, 'standard', 8)

In [5]:
good_eids_df = pd.read_csv('/s/project/deepMMR/data/fundus/processed_transferGWAS/good_eids.tsv', sep='\t')
good_eids_df['eid'] = good_eids_df['eid'].astype('str') + '_' + good_eids_df['instance'].astype('str')
good_eids_df

,Unnamed: 0,eid,instance
0,0,1000066_0_0,0_0
1,1,1000156_0_0,0_0
2,2,1000199_0_0,0_0
3,3,1000378_0_0,0_0
4,4,1000424_0_0,0_0
...,...,...,...
47854,47854,6002837_0_0,0_0
47855,47855,6003318_0_0,0_0
47856,47856,6003414_0_0,0_0
47857,47857,6003874_0_0,0_0


In [6]:
slurm_dir = '/data/ouga/home/ag_gagneur/graef/tmp/slurm/img_extraction'
dfs = []
for folder_name in tqdm(filter(lambda x: x.startswith('run_'), os.listdir(slurm_dir))):
    df = pd.read_parquet(os.path.join(slurm_dir, folder_name, 'resnet50_imagenet_L2.parquet'))
    print(df.shape)
    dfs.append(df)
df = pd.concat(dfs)

1it [00:05,  5.24s/it]

(10947, 8192)


2it [00:09,  4.47s/it]

(10947, 8192)


3it [00:12,  4.17s/it]

...

(10947, 8192)


7it [00:29,  4.27s/it]

(10947, 8192)


8it [00:33,  4.19s/it]

(10947, 8192)


In [7]:
print('Filtering for good images')
df = df[df.index.isin(good_eids_df['eid'])]

df.index = df.index.str.split('_').str[0]
df.index.names = ['eid']

duplicates = df.index.duplicated(keep='first')
df = df[~duplicates]

df.columns = [f'feature_{ii}' for ii in range(df.shape[1])]

df.to_parquet('/s/project/deepMMR/data/phenotypes/stage_1_raw/Fundus_21015_21016/transferGWAS_L2.parquet')
df

Filtering for good images


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_8182,feature_8183,feature_8184,feature_8185,feature_8186,feature_8187,feature_8188,feature_8189,feature_8190,feature_8191
eid,,,,,,,,,,,,,,,,,,,,,
4746235,-0.007287,0.007381,0.004827,-0.027325,-0.011589,-0.000532,-0.000636,-0.011887,-0.013781,-0.018992,...,-0.010468,0.002388,0.051467,-0.027536,-0.002665,0.003277,-0.008262,0.001201,0.035584,-0.017093
4746244,-0.007520,0.004948,0.003814,-0.027827,-0.008348,0.000156,-0.001515,-0.012842,-0.017247,-0.017575,...,-0.013473,0.001640,0.051003,-0.028293,-0.002202,0.004994,-0.008228,0.001291,0.036507,-0.018190
4746283,-0.007625,0.006463,0.003738,-0.027323,-0.010450,0.000060,-0.000631,-0.011341,-0.015308,-0.020838,...,-0.014633,0.002499,0.047495,-0.027243,-0.001892,0.003838,-0.006114,0.001633,0.035747,-0.016564
4746340,-0.008650,0.011590,0.003835,-0.025869,-0.008135,0.000299,-0.000156,-0.008941,-0.013452,-0.022442,...,-0.009872,0.004387,0.048270,-0.027720,-0.002532,0.005123,-0.006308,-0.000499,0.039992,-0.017220
4746523,-0.008541,0.010654,0.002708,-0.027680,-0.009490,0.000505,-0.001641,-0.010237,-0.015928,-0.021375,...,-0.012170,0.001979,0.051081,-0.026883,-0.002345,0.003785,-0.007320,0.000588,0.036885,-0.016694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4745681,-0.008686,0.009349,0.002736,-0.027978,-0.010846,0.000633,-0.001318,-0.011052,-0.015311,-0.018219,...,-0.009786,0.002376,0.051105,-0.027534,-0.002829,0.005602,-0.008163,0.000963,0.038570,-0.018903
4745711,-0.008271,0.012663,0.003532,-0.025016,-0.009126,0.001023,-0.000161,-0.008260,-0.013856,-0.015674,...,-0.010343,0.002724,0.052138,-0.026885,-0.002980,0.004976,-0.009757,0.000769,0.041677,-0.016508
4745817,-0.007684,0.005122,0.003274,-0.027725,-0.010537,0.000767,-0.000758,-0.011407,-0.015719,-0.018414,...,-0.013826,0.002009,0.047930,-0.027480,-0.002335,0.003495,-0.005412,0.001272,0.038117,-0.016350


In [8]:
pca = Sklearn_PCA(n_components=256)
    
transformed_data = pd.DataFrame(pca.fit_transform(df), index=df.index)

In [9]:
transformed_data.columns = [f'pca_{ii}' for ii in range(len(transformed_data.columns))]
transformed_data

,pca_0,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,...,pca_246,pca_247,pca_248,pca_249,pca_250,pca_251,pca_252,pca_253,pca_254,pca_255
eid,,,,,,,,,,,,,,,,,,,,,
4746235,-0.011358,-0.002083,0.077565,0.011951,-0.029286,-0.022704,0.043909,0.007249,0.010872,0.008214,...,0.001914,-0.002028,0.001487,0.002988,0.001665,-0.000488,0.001726,-0.002538,0.001224,0.000444
4746244,-0.071913,0.045298,-0.006511,0.043927,-0.017685,0.012145,0.017945,0.006497,0.021267,0.016547,...,0.000512,-0.000266,-0.001708,-0.000450,-0.001791,0.001555,0.002683,0.000326,-0.000685,-0.002209
4746283,-0.068800,-0.024368,-0.024997,0.012008,0.019040,0.012938,0.010016,-0.008356,0.033474,-0.020081,...,0.002255,-0.001019,-0.000330,0.001345,-0.000042,0.000075,-0.000010,-0.000860,0.001064,0.000104
4746340,0.193234,-0.016039,-0.037309,0.041395,0.046053,0.046231,0.023219,0.004413,-0.002440,-0.034607,...,0.000929,0.001368,0.000033,-0.000673,0.002683,0.000431,-0.001245,0.000168,-0.000241,-0.000099
4746523,-0.001638,-0.039193,-0.008635,-0.002439,-0.023769,-0.003056,0.007038,0.010050,-0.005627,-0.018344,...,-0.002491,0.000812,0.000041,0.000827,-0.000635,-0.002120,0.001354,-0.001240,-0.000851,-0.000835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4745681,-0.030991,0.041967,-0.008793,-0.066281,-0.023950,-0.020324,-0.003612,0.015451,0.015496,-0.003233,...,-0.000590,-0.000033,0.001868,-0.001526,0.000601,0.000225,-0.000889,0.001193,0.000581,-0.001900
4745711,0.142173,0.014922,0.099255,-0.026183,-0.054846,-0.052645,-0.027803,-0.013666,0.005466,-0.008652,...,-0.000862,0.000796,-0.000152,-0.002078,0.000383,-0.000128,0.000075,0.000721,-0.000105,-0.001409
4745817,-0.092807,-0.044321,-0.018683,-0.021904,0.014602,0.021432,0.012579,0.008555,0.027473,0.010990,...,0.000595,0.000514,0.000227,0.000197,-0.001496,-0.001120,0.000585,0.000613,0.000035,0.001540


In [10]:
transformed_data.iloc[:, range(10)].to_parquet('/s/project/deepMMR/data/phenotypes/stage_1_raw/Fundus_21015_21016/transferGWAS_L2_pca_10.parquet')
transformed_data.iloc[:, range(20)].to_parquet('/s/project/deepMMR/data/phenotypes/stage_1_raw/Fundus_21015_21016/transferGWAS_L2_pca_20.parquet')
transformed_data.to_parquet('/s/project/deepMMR/data/phenotypes/stage_1_raw/Fundus_21015_21016/transferGWAS_L2_pca_256.parquet')

# Evaluation - Pretrained ResNet50

In [ ]:
model = pretrained_pca.ImageEncoder('resnet50').to(device)
features_left, labels_left = pretrained_pca.extract_features(model, train_dataloader_left, device)
features_right, labels_right = pretrained_pca.extract_features(model, train_dataloader_right, device)

/opt/modules/i12g/anaconda/envs/deepMMR/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/modules/i12g/anaconda/envs/deepMMR/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Process inputs:  48%|████▊     | 42/88 [30:21<32:52, 42.89s/it] 

In [ ]:
data_left = {'_'.join(np.array(label.split('_'))[[0, 2, 3]]): feature for (label, feature) in zip(labels_left, features_left)}
data_right = {'_'.join(np.array(label.split('_'))[[0, 2, 3]]): feature for (label, feature) in zip(labels_right, features_right)}
data = {label: np.concatenate([feature_left, data_right[label]]) for (label, feature_left) in data_left.items() if label in data_right.keys()}
pca = Sklearn_PCA(n_components=128) # 10
embeddings = pca.fit_transform(np.stack(data.values()))
minimum_variance_explained = 0.8
pca_s = pca.explained_variance_ratio_[np.cumsum(pca.explained_variance_ratio_) < minimum_variance_explained]

embeddings = embeddings[:, range(len(pca_s))] 

df = pd.DataFrame(embeddings, index=data.keys(), columns=[f'pca_{num}' for num in range(embeddings.shape[1])])
df.to_parquet('/s/project/deepMMR/data/phenotypes/stage_1_raw/Fundus_21015_21016/resnet_50_pca_raw.parquet')

In [51]:
df = pd.read_parquet('/s/project/deepMMR/data/phenotypes/stage_1_raw/Fundus_21015_21016/resnet_50_pca_raw.parquet')
df.index = df.index.str.split('_').str[0]
df.index.names = ['eid']
duplicates = df.index.duplicated(keep='first')
df = df[~duplicates]
df.to_parquet('/s/project/deepMMR/data/phenotypes/stage_1_raw/Fundus_21015_21016/resnet_50_pca.parquet')
df

,pca_0,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,...,pca_41,pca_42,pca_43,pca_44,pca_45,pca_46,pca_47,pca_48,pca_49,pca_50
eid,,,,,,,,,,,,,,,,,,,,,
1000066,-5.446280,2.134828,-4.652717,-2.125299,0.580376,0.505437,3.645508,-1.465071,0.559960,-0.148222,...,0.353012,0.038454,0.205035,-0.641453,0.869113,0.092259,-0.582056,1.186516,-0.610516,-0.408618
1000156,-8.319563,-0.242977,0.330990,2.693621,0.226985,-0.173033,0.044034,0.841468,-0.099231,-0.782438,...,-1.260801,0.136870,0.088421,-0.792122,-0.942989,-0.917753,0.167208,-1.023388,-0.054315,0.599594
1000199,-7.162394,-0.387870,-1.151493,1.097107,3.034690,-2.266430,-0.223179,-0.279714,-0.669099,0.161026,...,-0.672399,-0.107862,0.902505,0.103369,0.715633,-0.988445,-0.291919,0.149660,-0.161655,-0.139666
1000301,7.118132,1.133620,1.888029,1.358146,0.974229,0.119056,2.924170,-3.068111,-3.560253,-1.671859,...,0.467257,1.129952,1.485040,-0.275029,-1.220480,-0.125073,0.981571,-0.995944,0.407328,-0.853552
1000378,-3.639200,-0.193422,-4.519847,-3.138984,4.326903,-0.346467,-1.569447,-1.438142,0.437637,0.478612,...,0.138112,0.243413,0.888183,0.624756,-0.050007,-0.380207,-0.709729,1.088905,0.890469,-0.371500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3823925,-4.455645,1.548255,1.318259,1.836927,-2.422237,-3.076377,1.413838,-1.080322,2.505049,-1.520907,...,0.088513,-0.649545,0.300166,-0.139134,0.258432,0.461719,-0.504943,0.859834,0.008174,-0.095109
3823932,-3.649822,2.648258,2.238149,6.375959,-3.792537,1.586219,0.554872,-1.598632,-1.841741,-1.482859,...,-0.415452,0.231903,-1.232324,0.119146,0.348567,0.501581,-0.113081,-0.491218,-0.075475,-0.585068
3823981,-1.584543,1.011757,6.513752,-1.810544,1.136091,2.039510,1.944103,-1.083197,2.520791,1.933523,...,0.062662,-0.135144,0.111591,-0.342395,-0.433655,0.361703,0.123229,-0.903594,-0.113467,0.142126


In [15]:
df = pd.read_parquet('/s/project/deepMMR/data/phenotypes/stage_1_raw/Fundus_21015_21016/resnet_50_pca.parquet')
df.index = df.index.str.split('_', expand=True)
df.index.names = ['eid', 'fid', 'instance_id', 'array_id']

In [16]:
df_collapsed = df.reset_index().groupby('eid').last()
df_collapsed = df_collapsed.drop(['fid', 'instance_id', 'array_id'], axis=1)
df_collapsed.to_parquet('/s/project/deepMMR/data/phenotypes/stage_1_raw/Fundus_21015_21016/resnet_50_pca.parquet')
df_collapsed

,pca_0,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9
eid,,,,,,,,,,
1000066,-2.252839,3.066601,-2.913292,-2.328768,1.500444,1.818484,-1.092669,-1.270074,-1.512550,3.528600
1000156,-5.963784,0.325908,-1.809769,2.433903,-0.624344,0.911276,-0.220119,0.139989,-0.655710,1.087488
1000199,-4.968379,-0.170561,-0.363203,1.222055,2.232448,-0.122127,-1.113042,-0.046429,-1.952182,0.913763
1000301,6.130622,1.322871,1.488849,-0.134390,1.098978,0.586573,-3.117746,-1.187186,0.740730,-0.114611
1000378,-4.354016,-0.258280,-2.338870,-0.358884,3.314417,1.889463,-0.674396,2.056707,-1.619832,-3.547954
...,...,...,...,...,...,...,...,...,...,...
6025959,2.996337,-0.728765,2.447221,-1.584065,-2.570893,-0.322580,-1.135632,0.223566,-0.775059,-1.589258
6026028,4.927389,3.239777,0.903430,4.418314,2.202457,2.384530,1.282864,-0.437195,-0.633742,0.861229
6026072,8.695135,-3.834220,-2.078501,2.316255,-0.579175,-1.012689,2.724683,0.646925,0.066713,-1.181772


## Train SimCLR

In [12]:
def train_linear_model(embeddings: torch.Tensor, labels: torch.Tensor, device) -> nn.Module:
    # Train a linear model
    linear_model = nn.Linear(embeddings.shape[1], len(torch.unique(labels))).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(linear_model.parameters(), lr=3e-4)

    # Training loop
    for epoch in range(5000):
        optimizer.zero_grad()
        outputs = linear_model(embeddings)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    return linear_model

def plot_roc_curve(labels: List[int], scores: List[float]):
    fpr, tpr, _ = roc_curve(labels, scores)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()

In [1]:
# Train a linear model
linear_model = train_linear_model(embeddings_tensor, labels_tensor, device)

# Calculate scores using the linear model
scores = nn.functional.softmax(linear_model(embeddings_tensor)).detach().cpu().numpy()
# Plot ROC curve for each class
for class_label in torch.unique(labels_tensor):
    class_scores = scores[:, class_label]
    class_labels = [1 if label == class_label else 0 for label in labels_tensor]
    plot_roc_curve(class_labels, class_scores)

NameError: name 'train_linear_model' is not defined

# Evaluation - Pretrained SimCLR

In [6]:
from pl_bolts.models.self_supervised import SimCLR
import pytorch_lightning as pl

# model
weight_path = 'https://pl-bolts-weights.s3.us-east-2.amazonaws.com/simclr/bolts_simclr_imagenet/simclr_imagenet.ckpt'
model = SimCLR.load_from_checkpoint(weight_path, strict=False)
model.to(device)
model.freeze()

def extract_features(model: nn.Module, dataloader: DataLoader, device: torch.device) -> np.ndarray:
    features = []
    labels = []
    model.eval()
    with torch.no_grad():
        for inputs, file_names in tqdm(dataloader, desc='Process inputs'):
            output = model(inputs.to(device))
            features.append(output.cpu().numpy())
            labels.extend([f.split('.')[0] for f in file_names])
    return np.concatenate(features, axis=0).squeeze(), labels

features_left, labels_left = extract_features(model, train_dataloader_left, device)
features_right, labels_right = extract_features(model, train_dataloader_right, device)

/opt/modules/i12g/anaconda/envs/deepMMR/lib/python3.8/site-packages/pl_bolts/__init__.py:11: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/opt/modules/i12g/anaconda/envs/deepMMR/lib/python3.8/site-packages/pl_bolts/__init__.py:11: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/opt/modules/i12g/anaconda/envs/deepMMR/lib/python3.8/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/opt/modules/i12g/anaconda/envs/deepMMR/lib

In [7]:
data_left = {'_'.join(np.array(label.split('_'))[[0, 2, 3]]): feature for (label, feature) in zip(labels_left, features_left)}
data_right = {'_'.join(np.array(label.split('_'))[[0, 2, 3]]): feature for (label, feature) in zip(labels_right, features_right)}
data = {label: np.concatenate([feature_left, data_right[label]]) for (label, feature_left) in data_left.items() if label in data_right.keys()}

df = pd.DataFrame(np.stack(data.values()), index=data.keys(), columns=[f'feature_{num}' for num in range(np.stack(data.values()).shape[1])])
df.to_parquet('/s/project/deepMMR/data/phenotypes/stage_1_raw/Fundus_21015_21016/sim_clr_raw.parquet')

/opt/modules/i12g/anaconda/envs/deepMMR/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3445: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if await self.run_code(code, result, async_=asy):


In [6]:
df_raw = pd.read_parquet('/s/project/deepMMR/data/phenotypes/stage_1_raw/Fundus_21015_21016/sim_clr_raw.parquet')

df_raw.index.names = ['eid']
df_raw.index = df_raw.index.str.split('_').str[0]
df_raw.index.names = ['eid']
duplicates = df_raw.index.duplicated(keep='first')
df_raw = df_raw[~duplicates]

pca = Sklearn_PCA(n_components=256) # 10
embeddings = pca.fit_transform(np.stack(df_raw.values))
df = pd.DataFrame(embeddings, index=df_raw.index, columns=[f'pca_{num}' for num in range(embeddings.shape[1])])
df.to_parquet('/s/project/deepMMR/data/phenotypes/stage_1_raw/Fundus_21015_21016/sim_clr_pca.parquet')

In [7]:
df_raw = pd.read_parquet('/s/project/deepMMR/data/phenotypes/stage_1_raw/Fundus_21015_21016/sim_clr_pca.parquet')
df = df_raw.iloc[:, range(10)]
df.to_parquet('/s/project/deepMMR/data/phenotypes/stage_1_raw/Fundus_21015_21016/sim_clr_pca_10.parquet')
df

,pca_0,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9
eid,,,,,,,,,,
1000066,-1.124531,0.476144,0.039057,0.037099,0.415730,0.050869,-0.071730,0.004496,-0.004564,0.515702
1000156,-1.246989,-0.333761,0.595704,-0.233015,-0.303992,-0.124384,0.152929,-0.035439,0.224985,-0.022164
1000199,-1.545924,0.019440,0.332943,0.215149,-0.147210,0.001268,-0.025074,-0.034308,-0.010866,-0.100134
1000301,1.464271,0.471742,0.117708,-0.446180,0.069569,-0.074620,-0.111770,-0.483076,0.171306,0.125713
1000378,-1.011024,-0.077450,0.054507,0.241299,-0.424258,0.304522,-0.272214,-0.117854,0.375444,-0.005743
...,...,...,...,...,...,...,...,...,...,...
3823925,-0.721995,-0.035558,-0.509513,-0.167597,-0.022988,-0.273178,-0.185490,0.045779,-0.054213,0.274207
3823932,0.062510,0.952701,0.346640,-0.476639,0.020480,0.344963,0.568104,-0.512069,0.072603,0.214979
3823981,0.236588,0.156204,-0.349229,-0.943248,-0.027832,-0.306307,-0.173657,0.403887,-0.052307,-0.143568


# Training - SimCLR